<a href="https://colab.research.google.com/github/ayesha-119/Deep-Learning-BWF/blob/master/Task_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🔴 Task 23**

**Topics: Dataset Loading and Understanding Training, Validation and Test Sets**

Resource:https://drive.google.com/file/d/1i9dPxM_1M4HYN5bYxFcuklC1vM0GrOCq/view?usp=share_link

This is a very important concept theoretically and it's implementation is also present in detail in the resource. Splitting data is a good proportion of subsets is critical

# **HOLD-OUT VALIDATION**

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import boston_housing

# Load the Boston Housing dataset
(x_train, y_train), (x_test, y_test) = boston_housing.load_data(test_split=0.2)

# Normalize the input features
x_mean = np.mean(x_train, axis=0)
x_std = np.std(x_train, axis=0)
x_train = (x_train - x_mean) / x_std
x_test = (x_test - x_mean) / x_std

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_val, y_val), verbose=1)

# Evaluate the model on the test set
predictions = model.predict(x_test)
test_loss = mean_squared_error(y_test, predictions)
print("Test Loss:", test_loss)


57026/57026 [==============================] - 0s 0us/step
Epoch 1/100
11/11 [==============================] - 1s 14ms/step - loss: 607.5773 - val_loss: 463.2588
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 586.1160 - val_loss: 448.0572
Epoch 3/100
11/11 [==============================] - 0s 4ms/step - loss: 565.4619 - val_loss: 432.8414
Epoch 4/100
11/11 [==============================] - 0s 5ms/step - loss: 544.0974 - val_loss: 417.3165
Epoch 5/100
11/11 [==============================] - 0s 3ms/step - loss: 522.4559 - val_loss: 401.5220
Epoch 6/100
11/11 [==============================] - 0s 3ms/step - loss: 500.2608 - val_loss: 385.0271
Epoch 7/100
11/11 [==============================] - 0s 3ms/step - loss: 477.4463 - val_loss: 367.6278
Epoch 8/100
11/11 [==============================] - 0s 4ms/step - loss: 452.3172 - val_loss: 349.3367
Epoch 9/100
11/11 [==============================] - 0s 5ms/step - loss: 427.1583 - val_loss: 330.7510
Epoch 10/100


# **K-FOLD CROSS-VALIDATION**

In [6]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import boston_housing

# Load the Boston Housing dataset
(x, y), _ = boston_housing.load_data(test_split=0.0)

# Normalize the input features
x_mean = np.mean(x, axis=0)
x_std = np.std(x, axis=0)
x = (x - x_mean) / x_std

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(x.shape[1],)))
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Perform k-fold cross-validation
k = 5  # Number of folds
fold_scores = []

kf = KFold(n_splits=k, shuffle=True, random_state=42)
for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Train the model
    model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
    
    # Evaluate the model on the validation set
    val_predictions = model.predict(x_val)
    fold_score = mean_squared_error(y_val, val_predictions)
    fold_scores.append(fold_score)

# Calculate the average score across all folds
avg_score = np.mean(fold_scores)
print("Average Score:", avg_score)


4/4 [==============================] - 0s 2ms/step
Average Score: 13.269621115582172


# **ITERATED K-FOLD VALIDATION WITH SHUFFLING**

In [8]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Generate a small synthetic dataset
x = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([0, 1, 0, 1, 0])

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(2,)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Perform iterated k-fold cross-validation with shuffling
k = 3  # Number of folds
num_iterations = 3  # Number of iterations
all_scores = []

for iteration in range(num_iterations):
    print("Iteration:", iteration + 1)
    
    fold_scores = []
    
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the model
        model.fit(x_train, y_train, epochs=10, batch_size=1, verbose=0)

        # Evaluate the model on the test set
        test_predictions = model.predict(x_test)
        test_predictions = (test_predictions > 0.5).astype(int)
        fold_score = accuracy_score(y_test, test_predictions)
        fold_scores.append(fold_score)

    all_scores.append(fold_scores)

# Calculate the average score across all iterations and folds
avg_scores = np.mean(all_scores, axis=0)
print("Average Scores:", avg_scores)


Iteration: 1
1/1 [==============================] - 0s 15ms/step
Iteration: 2
1/1 [==============================] - 0s 16ms/step
Iteration: 3
1/1 [==============================] - 0s 14ms/step
Average Scores: [0.5 0.5 0. ]
